In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [3]:
df_meal=pd.read_csv("DL_Meal.csv")

In [4]:
df_meal["meal_name_raw"].value_counts()

meal_name_raw
Boiled Lunch Balanced         148
Raw Lunch Low-Carb            147
Roasted Snack Vegetarian      144
Baked Snack Paleo             143
Baked Snack Vegetarian        140
                             ... 
Fried Lunch Paleo              96
Roasted Breakfast Balanced     96
Boiled Snack Vegetarian        94
Raw Snack Balanced             93
Fried Snack Balanced           89
Name: count, Length: 168, dtype: int64

In [3]:
feature_cols = [
    "Age", "Gender", "Weight (kg)", "Height (m)", "BMI",
    "Daily meals frequency",
    "Carbs", "Proteins", "Fats", "Calories",
    "Workout_Type", "Calories_Burned", "cal_balance",
    "goal_label"
]

X = df_meal[feature_cols].values
y_meal = df_meal["meal_type"].values
y_diet = df_meal["diet_type"].values
y_cook = df_meal["cooking_method"].values




In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, \
y_meal_train, y_meal_test, \
y_diet_train, y_diet_test, \
y_cook_train, y_cook_test = train_test_split(
    X,
    y_meal,
    y_diet,
    y_cook,
    test_size=0.2,
    random_state=42
)


In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.mean(axis=0))
print(X_train.std(axis=0))


[ 1.09362432e-14 -1.81799020e-17  1.21670174e-14  9.11337672e-15
 -4.81003085e-15 -8.47856334e-15 -2.71011685e-15 -2.49408966e-14
  9.21666216e-15 -1.75193193e-16  5.17988430e-16  2.03465716e-15
  1.00523756e-15 -8.80615025e-17]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [6]:
from torch.utils.data import Dataset,DataLoader
class MealDataset(Dataset):
    def __init__(self, X, y1, y2, y3):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y1 = torch.tensor(y1, dtype=torch.long)
        self.y2 = torch.tensor(y2, dtype=torch.long)
        self.y3 = torch.tensor(y3, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y1[idx], self.y2[idx], self.y3[idx]


In [7]:
train_dataset = MealDataset(
    X_train,
    y_meal_train,
    y_diet_train,
    y_cook_train
)


In [8]:
test_dataset = MealDataset(
    X_test,
    y_meal_test,
    y_diet_test,
    y_cook_test
)

In [ ]:
class MealRecommenderDL(nn.Module):
    def __init__(self, input_dim, meal_c, diet_c, cook_c):
        super().__init__()

        self.shared = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )

        self.meal_head = nn.Linear(64, meal_c)
        self.diet_head = nn.Linear(64, diet_c)
        self.cook_head = nn.Linear(64, cook_c)

    def forward(self, x):
        h = self.shared(x)
        return (
            self.meal_head(h),
            self.diet_head(h),
            self.cook_head(h)
        )


In [ ]:
        return (
            self.meal_head(h),
            self.diet_head(h),
            self.cook_head(h)
        )


In [10]:
meal_c=df_meal["meal_type"].nunique()
diet_c=df_meal["diet_type"].nunique()
cook_c=df_meal["cooking_method"].nunique()


In [11]:
model = MealRecommenderDL(
    input_dim=len(feature_cols),
    meal_c=meal_c,
    diet_c=diet_c,
    cook_c=cook_c
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [12]:
epochs = 10

for epoch in range(epochs):
    total_loss = 0
    for Xb, yb1, yb2, yb3 in train_dataset:

        o1, o2, o3 = model(Xb)

        loss = (
            criterion(o1, yb1) +
            criterion(o2, yb2) +
            criterion(o3, yb3)
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        avg_loss=total_loss/len(train_dataset)

    print(f"Epoch {epoch+1} | AVG Loss: {avg_loss:.3f}")


Epoch 1 | AVG Loss: 5.130
Epoch 2 | AVG Loss: 5.126
Epoch 3 | AVG Loss: 5.126
Epoch 4 | AVG Loss: 5.126
Epoch 5 | AVG Loss: 5.126
Epoch 6 | AVG Loss: 5.126
Epoch 7 | AVG Loss: 5.126
Epoch 8 | AVG Loss: 5.126
Epoch 9 | AVG Loss: 5.126
Epoch 10 | AVG Loss: 5.126


In [13]:
def evaluate(model, dataloader):
    model.eval()

    correct_meal = correct_diet = correct_cook = 0
    total = 0

    with torch.no_grad():
        for Xb, yb1, yb2, yb3 in dataloader:
            o1, o2, o3 = model(Xb)

            correct_meal += (o1.argmax(1) == yb1).sum().item()
            correct_diet += (o2.argmax(1) == yb2).sum().item()
            correct_cook += (o3.argmax(1) == yb3).sum().item()

            total += yb1.size(0)

    print("Meal Type Accuracy:", correct_meal / total)
    print("Diet Type Accuracy:", correct_diet / total)
    print("Cooking Method Accuracy:", correct_cook / total)


In [14]:
evaluate(model, DataLoader(test_dataset, batch_size=64))


Meal Type Accuracy: 0.247
Diet Type Accuracy: 0.152
Cooking Method Accuracy: 0.147


In [15]:
def recommend_meals(
    user_input,
    meal_df,
    model,
    scaler,
    top_n=3
):

    bmi = user_input["weight"] / (user_input["height"] ** 2)
    model.eval()
    results = {}
    

    # encode goal ONCE
    goal_map = {
        "weight loss": 0,
        "weight gain": 1
        }
    goal_encoded = goal_map[user_input["goal"]]

    meal_types = ["Breakfast","Lunch","Dinner","Snack"]
    for meal_name in meal_types:

        meal_candidates = meal_df[meal_df["meal_type_raw"] == meal_name]
        recommendations = []

        for _, row in meal_candidates.iterrows():

            X = [
                user_input["age"],
                user_input["gender"],          # encoded
                user_input["weight"],
                user_input["height"],
                bmi,
                row["Daily meals frequency"],
                row["Carbs"],
                row["Proteins"],
                row["Fats"],
                row["Calories"],
                row["Workout_Type"],           # encoded
                row["Calories_Burned"],
                row["cal_balance"],
                goal_encoded
            ]

            X = scaler.transform([X])
            X_t = torch.tensor(X, dtype=torch.float32)

            with torch.no_grad():
                meal_o, diet_o, cook_o = model(X_t)

                score = (
                    torch.softmax(meal_o, dim=1).max().item() +
                    torch.softmax(diet_o, dim=1).max().item() +
                    torch.softmax(cook_o, dim=1).max().item()
                )
               
                recommendations.append({
                    "meal_type": meal_name,  # already string
                    "meal_name":row["meal_name_raw"],
                    "diet_name" : row["diet_type_raw"],
                    "cooking_method" :row["cooking_method_raw"],
                    "score": score
                })

        recommendations.sort(key=lambda x: x["score"], reverse=True)
        results[meal_name] = recommendations[:top_n]
        rows = []

        for meal_type, meals in results.items():
            for meal in meals:
                rows.append(meal)
        
        results_df = pd.DataFrame(rows)
    return results_df


In [16]:
user_input = {
    "age": 30,
    "weight": 70,
    "height": 1.65,
    "gender": 1  , # male
    "goal":"weight loss",
    "preferred_diet":["Vegan","Vegetarian"]
}


In [32]:
preferred_diet = user_input["preferred_diet"]
print(preferred_diet)
filtered_meals = df_meal[df_meal["diet_type_raw"].isin(preferred_diet)]



['Vegan', 'Vegetarian']


In [33]:
top_meals = recommend_meals(
    user_input,
    meal_df=filtered_meals,
    model=model,
    scaler=scaler,
    top_n=3
)

In [34]:
top_meals

,meal_type,meal_name,diet_name,cooking_method,score
0,Breakfast,Baked Breakfast Vegan,Vegan,Baked,0.580479
1,Breakfast,Fried Breakfast Vegetarian,Vegetarian,Fried,0.580479
2,Breakfast,Fried Breakfast Vegan,Vegan,Fried,0.580479
3,Lunch,Grilled Lunch Vegan,Vegan,Grilled,0.580479
4,Lunch,Fried Lunch Vegetarian,Vegetarian,Fried,0.580479
5,Lunch,Boiled Lunch Vegan,Vegan,Boiled,0.580479
6,Dinner,Baked Dinner Vegan,Vegan,Baked,0.580479
7,Dinner,Boiled Dinner Vegan,Vegan,Boiled,0.580479
8,Dinner,Baked Dinner Vegetarian,Vegetarian,Baked,0.580479
9,Snack,Boiled Snack Vegan,Vegan,Boiled,0.580479


In [21]:
checkpoint = {
    "model_state_dict": model.state_dict(),
    "input_dim": X.shape[1],
    "meal_c": meal_c,
    "diet_c": diet_c,
    "cook_c": cook_c,
    "feature_cols": feature_cols,          # VERY IMPORTANT
    "scaler": scaler,   
    
}
torch.save(checkpoint, "meal_recommender.pth")